# Genre Prediction Model

In [1]:
import nltk
from nltk.corpus import stopwords
import re
import string
import pandas as pd
import os

## Configuração

In [19]:
# Execute na primeira vez em que usar este notebook
#nltk.download()
MODEL = os.path.join('..','app','model')
TRAINSET = os.path.join('.','train')
WORD_FEATURES = 2500
TEST_RATIO = 0.2 # Size of test set
RANDOM_SEED = 1010011010
N_TREES = 100 # Number of trees for RF
CORES = 3 # How many cores to user for parallel RF (-1 uses all cores)

## Pré processamento

Primeiramente carregamos as letras de música. Cada entrada no **train set** será formada por um verso.

In [84]:
# Descobre generos que deverão ser utilizados
genres = list()
for file in os.listdir(TRAINSET):
    if file.endswith(".csv"):
        genres.append(file[0:-4])

# Aqui salvaremos o train set sendo gerado a partir dos arquivos texto
raw_corpus = pd.DataFrame()

# Le os arquivos
for g in genres:
    df = pd.read_csv(os.path.join(TRAINSET,'{}.csv'.format(g)))
    df['genero'] = g
    raw_corpus = raw_corpus.append(df)

# Dados gerais
n_musicas = len(raw_corpus)
n_generos = len(genres)

# Quebra uma estrofe por linha
raw_corpus = raw_corpus.reset_index(drop=True)
raw_corpus['idx'] = range(len(raw_corpus))
s = raw_corpus['lyric'].str.split('\n').apply(pd.Series, 1).stack()
s.index = s.index.droplevel(-1) # to line up with df's index
s.name = 'estrofe'
raw_corpus = raw_corpus.join(s)
raw_corpus = raw_corpus[raw_corpus['estrofe'] != ' ']
raw_corpus = raw_corpus.reset_index(drop=True)

print("Corpus contém {} músicas em {}, com um total de {} estrofes".format(n_musicas, n_generos, len(raw_corpus)))

Corpus contém 3200 músicas em 4, com um total de 99380 estrofes


Agora iremos preparar as estrofes com as seguintes ações:

* remover stopwords;
* remover pontuação;
* garantindo que todas as palavras estejam em caixa baixa;
* quebrar cada estrofe em uma lista de palavras

In [9]:
def prepara_estrofes(frase, translator = str.maketrans('', '', string.punctuation)):
    # Remove pontuação
    frase = frase.translate(translator)
    
    # Garante que todas as palavras estão em caixa baixa
    palavras = frase.lower().split()
    
    # Lista de stopwords que usaremos
    stop_w = set(stopwords.words("portuguese"))
    
    # Retorna apenas as palavras que não são stopwords
    palavras_filtradas = [palavra for palavra in palavras if palavra not in stop_w]
    
    # Retorna a estrofe preparada
    return(" ".join(palavras_filtradas))

In [10]:
print(prepara_estrofes('Eu sei que !!! , )vou te amar'))

sei vou amar


Agora iremos aplicar nosso procedimento de preparação no nosso dataset

In [90]:
# Vamos salvas as estrofes preparadas nesta lista
estrofes_preparadas = list()
no_estrofes = len(raw_corpus)

for i in range(0, no_estrofes):
    if( (i+1)%10000 == 0 ):
        print("Verso %d de %d processado" % ( i+1, no_estrofes ) )
    estrofes_preparadas.append( prepara_estrofes(raw_corpus['estrofe'][i]) )
raw_corpus['versos_preparados'] = estrofes_preparadas

Verso 10000 de 99380 processado
Verso 20000 de 99380 processado
Verso 30000 de 99380 processado
Verso 40000 de 99380 processado
Verso 50000 de 99380 processado
Verso 60000 de 99380 processado
Verso 70000 de 99380 processado
Verso 80000 de 99380 processado
Verso 90000 de 99380 processado


## Modelo

Usaremos a abordagem **Bag of Words** para vetorizar nossas estrofes e treinar o modelo

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score
from sklearn.externals import joblib

In [54]:
def validacao(X, y):
    # TODO Mostrar gráficamente a confusion matrix
    print('Matriz de Confusão: \n{}'.format(confusion_matrix(y, X)))
    print('Accuracy: {}'.format(accuracy_score(y, X)))
    print('Precision: {}'.format(precision_score(y, X, average='macro')))
    print('Recall: {}'.format(recall_score(y, X, average='macro')))

In [15]:
vetorizador = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None,
                              stop_words = None, max_features = WORD_FEATURES)
train_data_features = vetorizador.fit_transform(estrofes_preparadas)
train_data_features = train_data_features.toarray()

In [18]:
X_train, X_test, y_train, y_test = train_test_split(train_data_features, raw_corpus['genero'],
                                                    test_size=TEST_RATIO, random_state=RANDOM_SEED)

In [21]:
# Inicialzando classificador e treinando modelo
forest = RandomForestClassifier(n_estimators = N_TREES, n_jobs=CORES, verbose=10) 
%time forest = forest.fit( X_train, y_train )
predictions_RF = forest.predict(X_test)

building tree 2 of 100building tree 3 of 100building tree 1 of 100


building tree 4 of 100


[Parallel(n_jobs=3)]: Done   2 tasks      | elapsed:   25.9s


building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100


[Parallel(n_jobs=3)]: Done   7 tasks      | elapsed:  1.3min


building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100


...

[Parallel(n_jobs=3)]: Done  92 tasks      | elapsed: 15.2min


building tree 95 of 100
building tree 96 of 100
building tree 97 of 100
building tree 98 of 100
building tree 99 of 100
building tree 100 of 100


[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed: 16.4min finished


CPU times: user 45min 23s, sys: 11.6 s, total: 45min 35s
Wall time: 16min 28s


[Parallel(n_jobs=3)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=3)]: Done   7 tasks      | elapsed:    0.2s
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:    0.4s
[Parallel(n_jobs=3)]: Done  19 tasks      | elapsed:    0.6s
[Parallel(n_jobs=3)]: Done  26 tasks      | elapsed:    0.8s
[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:    1.1s
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    1.3s
[Parallel(n_jobs=3)]: Done  55 tasks      | elapsed:    1.7s
[Parallel(n_jobs=3)]: Done  66 tasks      | elapsed:    2.0s
[Parallel(n_jobs=3)]: Done  79 tasks      | elapsed:    2.3s
[Parallel(n_jobs=3)]: Done  92 tasks      | elapsed:    2.7s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    2.9s finished


## Validação

In [53]:
# Validando por verso
validacao(predictions_RF, y_test)

Matriz de Confusão: 
[[2815  391  378  495]
 [ 780 4646  352  558]
 [ 559  308 3469  432]
 [ 696  648  450 2899]]
Accuracy: 0.6957637351579795
Precision: 0.6907750928335938
Recall: 0.6921693965476496


In [91]:
# Validando por música
raw_corpus['pred_verso'] = forest.predict(vetorizador.transform(raw_corpus['versos_preparados']))

[Parallel(n_jobs=3)]: Done   2 tasks      | elapsed:    0.3s
[Parallel(n_jobs=3)]: Done   7 tasks      | elapsed:    0.6s
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:    1.0s
[Parallel(n_jobs=3)]: Done  19 tasks      | elapsed:    1.5s
[Parallel(n_jobs=3)]: Done  26 tasks      | elapsed:    2.1s
[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:    2.8s
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    3.5s
[Parallel(n_jobs=3)]: Done  55 tasks      | elapsed:    4.4s
[Parallel(n_jobs=3)]: Done  66 tasks      | elapsed:    5.2s
[Parallel(n_jobs=3)]: Done  79 tasks      | elapsed:    6.3s
[Parallel(n_jobs=3)]: Done  92 tasks      | elapsed:    7.3s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    8.0s finished


In [101]:
from collections import Counter
predicoes_musica = list()
genero_musica = list()
for i in raw_corpus['idx'].drop_duplicates():
    df = raw_corpus[raw_corpus.idx == i]
    
    # Encontra classe mais comum
    count = Counter(df['pred_verso'])
    genero_musica.append(df['genero'].unique())
    predicoes_musica.append(count.most_common(1)[0][0])

In [104]:
# Predição por Música
validacao(predicoes_musica,genero_musica)

Matriz de Confusão: 
[[796   1   0   3]
 [  5 793   1   1]
 [  3   1 795   1]
 [  2   2   0 796]]
Accuracy: 0.99375
Precision: 0.9937689382178871
Recall: 0.99375


In [108]:
# Exportando modelo e vetorizador
joblib.dump(vetorizador, os.path.join(MODEL,'vetorizador.pkl.bz2'))
joblib.dump(forest, os.path.join(MODEL,'modelo.pkl.bz2')) 
joblib.dump(stopwords.words(), os.path.join(MODEL,'stopwords.pkl.bz2')) 

['../app/model/stopwords.pkl.bz2']